In [ ]:
!pip3 install torch torchvision torchaudio openai diffusers["torch"] transformers -q

In [ ]:
from openai import OpenAI
import os
import json
from diffusers import DiffusionPipeline
import torch



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
# Part for generating a prompts

In [ ]:
!pip3 install torch torchvision torchaudio openai diffusers["torch"] transformers -qc

In [ ]:
import json
from openai import OpenAI
api_key = ''
client = OpenAI()

# Opening JSON file
f = open('result.json')

# returns JSON object as
# a dictionary
data = json.load(f)

In [ ]:
def transform_input(input_string):
    # Split the input string into parts based on the '][' delimiter
    parts = input_string.split('"]["')

    # The first part (text excerpts) is further split by "','"
    # We remove the initial "['" and final "']" from the parts to clean up the strings
    texts = parts[0][2:].split('","')

    # The second part (prompts) is also split by "','", with similar cleanup
    prompts = parts[1][:-2].split('","')

    return texts, prompts

def generate_content(topic, issue):

  system_content = '''
  Stworz 'n' slajdów na ten temat podany przez użytkownika, gdzie 'n' to liczba podana przez użytkownika. Każdy slajd powinien mieć 1 zdanie. Użyj od 15 do 30 słów w zdaniu, zależnie od tego jak dużo materiału jest dostępne. Jeżeli w tekście jest jakieś wspomnienie o historycznej postaci, kraju lub data to użyj formatu: Z inicjatywy <person>Mieszka I</person> <country>Polska</country> przyjęła chrzest w <date>966 p.n.e </date>r. Używaj tylko tych trzech typów. Do każdego zdania wygeneruj prompt do obrazka, który powinien się znależć na prezentacji do danego zdania w jezyku angielskim. Prompt musi być generyczny, nie może przedstawiać konkretnych osób i nie może zawierać map. Może być powiązany z generalnymi osobami jak np ‘British army’ ‘polish people’, lub ‘german person’, może zawierać lokalizacje (np zamki, chiński mur, wieża Eiffla) i tym podobne. Prompt musi miec maksymalnie 15 słów. Prompt może zawierać szczegóły, ale nie ma mieć formatu opisowego. Ma konkretnie przedstawiać co powinno znaleźć się na obrazku bez dodatkowych słów opisujących na przykład znaczenie danego symbolu. Przykładowe prompt to: a polish medieval warrior sitting on a horse charging on the medieval multiple Swedish pikemen, oil painting, expressionism, detailed
  Przykład:
  Prompt użytkownika:
  Belle Époque (1871-1914) ,liczba slajdów: 5"
  Twój output:
  ["content_slajd1","content_slajd2","content_slajd3","content_slajd4","content_slajd5"]["prompt_slajd1","prompt_slajd2","prompt_slajd3","prompt_slajd4","prompt_slajd5"]
  przykład: ["Rewolucja neolityczna to przełomowy okres w historii ludzkości, rozpoczynający się około 12 000 lat temu, gdy myśliwi-zbieracze zaczęli osiadać i rozwijać rolnictwo, co zmieniło ich tryb życia i społeczeństwa","W wyniku rozwoju rolnictwa, ludzie zaczęli uprawiać ziemię, hodować zwierzęta i osiedlać się w miejscach stałych, co zapoczątkowało powstanie wczesnych osad rolniczych, takich jak Çatalhöyük w Anatolii i Jericho na Bliskim Wschodzie","Z inicjatywy osad rolniczych, zaczęło się wykształcać społeczeństwo złożone, gdzie ludzie specjalizowali się w różnych zawodach, a także powstawały pierwsze formy hierarchii społecznej i podziału pracy","Postęp technologiczny okresu neolitu, m.in. wynalezienie ceramiki i narzędzi z kamienia, miał istotny wpływ na rozwój gospodarczy i kulturowy społeczności, umożliwiając bardziej efektywne wykorzystanie zasobów i produkcję dóbr","Rewolucja neolityczna nie tylko zmieniła sposób bycia i gospodarkę ludzi, lecz także legła podstawy do przyszłych cywilizacji, stanowiąc istotny krok w kierunku rozwoju miast, pismem, i zaawansowanych form polityki i religii"]["Prehistoric hunters-gatherers settling, planting, and harvesting, transitioning to sedentary lifestyle, illustrated","Early agricultural settlements, such as Çatalhöyük and Jericho, with cultivated fields and domesticated animals, artistic depiction","Artisans and labor specialization in developing early social hierarchy, tools and ceramics visible, symbolic representation","Technological advancements in the Neolithic Era, with stone tools and pottery, enhancing economic activities, abstract symbols of progress","Foundations of future civilizations: abstract concept of cityscape, writing symbols, and diverse community roles, illustrating societal development"]

  '''

  user_prompt = f"{topic} - {issue} ,liczba slajdów: {5}"


  completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:huks::8vMuzg3f",
    temperature = 0,
    messages=[
      {"role": "system", "content": system_content},
      {"role": "user", "content": user_prompt}
    ]
  )
  output = completion.choices[0].message.dict().get('content')
  print(output)
  texts, prompts = transform_input(output)

  return texts, prompts

In [ ]:
f"{data[0].get('topic')} + {data[0].get('issues')}"

In [ ]:
data[0].get('issues')

In [ ]:
for records in data:
  topic = records.get('topic')
  issues = records.get('issues')
  contents_list = []
  prompts_list = []
  for issue in issues:
    transformed_texts, transformed_prompts = generate_content(topic, issue)
    contents_list.append(transformed_texts)
    prompts_list.append(transformed_prompts)
  records['content'] = contents_list
  records['prompts'] = prompts_list


In [ ]:
# New prompt test

In [ ]:
whole_dict = {}
for record in data:
    new_prompt = """
    Użytkownik poda Ci temat oraz listę lekcji do wygenerowania z danego tematu. Do każdej lekcji  stworz 'n' slajdów, gdzie 'n' to liczba slajdów podana przez użytkownika. Każdy slajd powinien mieć 1 zdanie. Użyj od 15 do 30 słów w zdaniu, zależnie od tego jak dużo materiału jest dostępne. Jeżeli w tekście jest jakieś wspomnienie o historycznej postaci, kraju lub data to użyj formatu: Z inicjatywy <person>Mieszka I</person> <country>Polska</country> przyjęła chrzest w <date>966 p.n.e </date>r. Używaj tylko tych trzech typów. Do każdego zdania wygeneruj 2 prompty do obrazków, które powinny się znależć na prezentacji do danego zdania w jezyku angielskim. Prompt musi być generyczny, nie może przedstawiać konkretnych osób i nie może zawierać map. Może być powiązany z generalnymi osobami jak np ‘British army’ ‘polish people’, lub ‘german person’, może zawierać lokalizacje (np zamki, chiński mur, wieża Eiffla) i tym podobne. Prompt musi miec maksymalnie 20 słów, ale nie mniej niż 3. Prompt może zawierać szczegóły, ale nie ma mieć formatu opisowego. Ma konkretnie przedstawiać co powinno znaleźć się na obrazku bez dodatkowych słów opisujących na przykład znaczenie danego symbolu. Przykładowe prompt to: a polish medieval warrior sitting on a horse charging on the medieval multiple Swedish pikemen, oil painting, expressionism, detailed.

    Wyniki zwróć w formacie json:
    Przykład:
    User input:'Temat: Starożytny Bliski Wschód + lista lekcji: ['Rewolucja neolityczna', 'Uwarunkowania geograficzne rozwoju cywilizacji na Bliskim Wschodzie', 'Organizacja państw i struktura społeczeństw w cywilizacjach starożytnego Bliskiego Wschodu', 'Najważniejsze osiągnięcia kulturowe starożytnych cywilizacji Bliskiego Wschodu', 'Wierzenia ludów tworzących cywilizację starożytnego Bliskiego Wschodu, z uwzględnieniem judaizmu'] liczba slajdów : 5':
    Your Output:
    {
        "slajd_name": "Rewolucja neolityczna",
        "content": [
          "Rewolucja neolityczna, znana również jako rewolucja rolnicza, zapoczątkowała przejście od zbieractwa i łowiectwa do rolnictwa i osiadłego trybu życia.",
          "Z inicjatywy tego procesu, ludzie zaczęli domestykować zwierzęta takie jak owce, kozy i bydło, co znacznie zmieniło ich sposób życia.",
          "Budowa trwałych osiedli i wiosek była możliwa dzięki stałym dostępom do żywności, co doprowadziło do rozwoju pierwszych społeczności rolniczych.",
          "Rewolucja neolityczna przyczyniła się do powstania nowych technologii, takich jak ceramika, tkactwo i wczesne formy irygacji.",
          "Zmiany te pozwoliły na wzrost liczby ludności i złożoność społeczeństw, co ostatecznie doprowadziło do powstania pierwszych cywilizacji."
        ],
        "prompts": [
          ["Neolithic farming, early agricultural activities", "Ancient people cultivating land, simple tools in use"],
          ["Domestication of sheep and goats in ancient times", "Early human settlers with domesticated cattle"],
          ["Ancient Neolithic village, mud houses", "Neolithic farmers working together, communal living"],
          ["Early Neolithic pottery making, clay vessels", "Ancient weaving, basic loom in operation"],
          ["Growing ancient population, community gatherings", "Complex Neolithic society, varied activities and roles"]
        ]
      }
    """

    user_prompt = f"Temat: {record.get('topic')} + lista lekcji: {record.get('issues')} liczba slajdów : 5"


    completion = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-1106:huks::8vMuzg3f",
    temperature = 0,
    messages=[
        {"role": "system", "content": new_prompt},
        {"role": "user", "content": user_prompt}
      ]
    )
    output = completion.choices[0].message.dict().get('content')
    print(output)
    json_output = json.loads(output)
    whole_dict[record.get('topic')] = json_output

In [ ]:
whole_dict